# Linked Art Data Visualisation - Timeline   


This notebook will demonstrate how to create a timeline data visualisation from Linked Art data files. It will first use collection data about John Ruskin's artworks, and then allow you to go through the process with your own collection data transformed to Linked Art JSON-LD.

<img src='media/vis.png' width="600"/>


## Timeline Data Visualisation

The [timeline data visualisation](https://knightlab.northwestern.edu/projects/) code from KnightLab will be used. 

Steps:
- read in Linked Art JSON-LD files
- parse file to create JSON input file for visualisation
- write JSON file to disc
- read in JSON file to KnightLab timeline visualisation

## Collection Data
The Linked Art JSON-LD files for John Ruskin's artworks were created with other notebooks:
- [Ruskin Transformation notebook](https://github.com/tgra/Linked-Art/blob/main/01-06-Transform-John-Ruskin.ipynb)
- [Ruskin Reconciliation notebook](https://github.com/tgra/Linked-Art/blob/main/02-01-Reconcile-John-Ruskin-Place-Names.ipynb)


In [1]:
try:
    import IPython
except:
    %pip install IPython
    import IPython 
from IPython.core.display import HTML
from IPython.display import display, IFrame, HTML

import os
import csv

try:
    import json
except:
    %pip install json
    import json 
    

## Read in Linked Art & transform to timeline

In [2]:
la_files = "./data/example/vis/input/"
file_list=os.listdir(r"./data/example/vis/input/")

In [3]:
sources = {}
sources["https://www.nga.gov/collection/"] = {
                "name":"National Gallery of Art",
                "colour": "#E8A798"}
sources["https://www.harvardartmuseums.org/collections/object/"] = {
                    "name":"Harvard Art Museum",
                    "colour":"#00758F"}
sources["ashmolean"] = {
                    "name":"Ashmolean Museum",
                    "colour":"#3f83e8"}
 
json_vis = {"events":[]} 
json_vis["title"] = {
    "media": {       
        "url": "//api.nga.gov/iiif/49a6128c-8d5a-4b00-beb3-36b29b97c0a1/full/!500,500/0/default.jpg",      
        "caption": "John Ruskin",      
        "credit": "Wikipedia"      } ,
        "text": {      
            "headline": "John Ruskin",      
            "text": "<p>This timeline visualisation shows artworks created by John Ruskin.</p><p>It demonstrates how the Linked Art data model can be used to transform, reconcile and visualise collections data for artworks.</p> <p>See https://linked.art for more information.</p>"
        } }
json_vis["eras"] = [
        {       
    "start_date": {   "year": "1819"   },   
    "end_date":{   "year": "1900"  },  
    "text": {"headline":"John Ruskin's lifetime"   }}
    ]

for file in file_list:
    # read file and append to 
    with open(la_files + file) as json_file:
        artwork = json.load(json_file)
        
        if "produced_by" not in artwork:
            continue
        if "begin_of_the_begin" not in artwork["produced_by"]["timespan"]:
            continue
        if artwork["produced_by"]["timespan"]["begin_of_the_begin"] in  (1819,""):
            continue
    
        id = artwork["id"]
        credit = ""
        text = ""
        imageurl = ""
        bgcolour = ""
        homepage = ""
  
        for source in list(sources.keys()):
            if source in id:
                credit = sources[source]["name"]
                bgcolour = sources[source]["colour"]
            
        if "referred_to_by" in artwork and len(artwork["referred_to_by"]) > 0 and "content" in artwork["referred_to_by"][0]:  
            text = artwork["referred_to_by"][0]["content"]
            
        try:
            if artwork["subject_of"][0]["classified_as"][1]["id"] == "http://vocab.getty.edu/aat/300266277":
                homepage = artwork["subject_of"][0]["id"]
                text = text + "<br/><br/><a target='_new' href='" + homepage+ "'>Artwork homepage</a>"
        except:
                 homepage = ""
                
        if artwork["representation"][0]["id"] != "":
            imageurl = artwork["representation"][0]["id"]
            if imageurl =="":
                continue
   
        if "begin_of_the_begin" in artwork["produced_by"]["timespan"]:
            begin = artwork["produced_by"]["timespan"]["begin_of_the_begin"]
        try:
            begin = int(begin)
        except:
            begin = ""
    
        if "end_of_the_end" in artwork["produced_by"]["timespan"]:
            end = artwork["produced_by"]["timespan"]["end_of_the_end"]
        try:
            end = int(end)
        except:
            end = ""
            
        json_vis["events"].append({
            "media": {        "url": imageurl,      "caption": "",      "credit": credit,      "thumbnail": imageurl        },
            "display_date": artwork["produced_by"]["timespan"]["_label"],
            "start_date": {          "year": begin },        
            "end_date": {         "year": end      },
            "text": {       "headline": artwork["_label"],      "text":  text      } ,
            "type": "title",
            "background":{"color":bgcolour}
        })

text_file = open('./data/example/vis/timeline.json', 'wt')  
n = text_file.write(json.dumps(json_vis,indent=2))
text_file.close()

## Timeline Using Local JSON File

The following HTML code shows how to publish a timeline using the JSON file created above.

In [4]:
%%HTML
        <link title="timeline-styles" rel="stylesheet"   href="https://cdn.knightlab.com/libs/timeline3/latest/css/timeline.css">
        <script src="https://cdn.knightlab.com/libs/timeline3/latest/js/timeline.js"></script>
        <div id='timeline-embed' style="width: 100%; height: 600px"></div>
        <script type="text/javascript">   window.timeline = new TL.Timeline('timeline-embed', "./data/example/vis/timeline.json");  </script>

# Now Your Turn

## Define Properties for Collection Data Source
Edit the example code below. If the artwork's identifier contains one of the sources key strings e.g. "https://www.nga.gov/collection/", the corresponding properties will be used to 
- add a label for the the artwork's source , and
- apply a background colour to the relevant 'page' of the timeline.

In [5]:
sources = {}
sources["STRING_IN_ID"] = {"name":"LABEL_ARTWORK_SOURCE","colour": "HEX_COLOUR"}
sources[""] = {"name":"LABEL_ARTWORK_SOURCE","colour": "GREEN"}

# example
#sources["harvardartmuseums"] = {"name":"harvard art ","colour": "blue"}


## Title Page

In [6]:
json_vis = {"events":[]} 

TITLE_PAGE_IMAGE_URL = "https://title_page_image.jpg"
TITLE_PAGE_IMAGE_CAPTION = "TITLE PAGE IMAGE CAPTION"
TITLE_PAGE_IMAGE_CREDIT = "TITLE PAGE IMAGE CREDIT"
TITLE_PAGE_HEADLINE = "TITLE PAGE HEADLINE"
TITLE_PAGE_TEXT = "TITLE PAGE TEXT"

json_vis["title"] = {
    "media": {       "url": TITLE_PAGE_IMAGE_URL,      "caption": TITLE_PAGE_IMAGE_CAPTION,      "credit": TITLE_PAGE_IMAGE_CREDIT      } ,
        "text": {      "headline": TITLE_PAGE_HEADLINE,      "text": " "
        }  
}

## Eras

In [7]:
ERA_START_DATE_YEAR = "1900"
ERA_END_DATE_YEAR = "2022"
ERA_TEXT_HEADLINE = "Timeline"

json_vis["eras"] = [{      
                    "start_date": {   "year": ERA_START_DATE_YEAR   },  
                    "end_date":{   "year": ERA_END_DATE_YEAR  }, 
                    "text": { "headline": ERA_TEXT_HEADLINE   }
}]


The following steps will take you through creating the timeline visualisation with your own collection of Linked Art files.

## Data File Directory
Specify the data file directory relative to the workshop directory that contains this notebook

In [8]:
la_files = "./data/working/linkedart_files/"
file_list=os.listdir(r"./data/working/linkedart_files/")

## Iterate through Linked Art files 

In [9]:

# image properties
media_url = ""
media_caption = ""
media_credit = ""
media_thumbnail = ""

#dates 
display_date = ""
start_date_year = ""
end_date_year = ""

#text
text_headline = ""
text_text = ""

# background colour
background_colour = ""

event = {
            "media": {  "url":media_url,      "caption": media_caption,      "credit": media_credit,      "thumbnail": media_thumbnail },
            "display_date": display_date,
            "start_date": {          "year": start_date_year },        
            "end_date": {         "year":  end_date_year     },
            "text": {       "headline": text_headline,      "text":  text_text     } ,
            "type": "title",
            "background":{"color":background_colour}
        }

for file in file_list:
    # image properties
    media_url = ""
    media_caption = ""
    media_credit = ""
    media_thumbnail = ""

#dates 
    display_date = ""
    start_date_year = ""
    end_date_year = ""

#text
    text_headline = ""
    text_text = ""

# background colour
    background_colour = ""

    # read file and append to 
    with open( la_files + file) as json_file:
        artwork = json.load(json_file)
        
        # do not include artwork in vis if produced_by date not available
        if "produced_by" not in artwork:
            continue
        if "begin_of_the_begin" not in artwork["produced_by"]["timespan"]:
            continue
        if artwork["produced_by"]["timespan"]["begin_of_the_begin"] in  (""):
            continue

        # image properties
        media_url = ""
        media_thumbnail = ""
        if artwork["representation"][0]["id"] != "":
            media_url =  artwork["representation"][0]["id"]
            media_thumbnail =  artwork["representation"][0]["id"]
        if media_url =="":
            continue
            
        for source in list(sources.keys()):
            if source in artwork["id"]:
                media_credit = sources[source]["name"]
                background_colour  = sources[source]["colour"]
                
        if "referred_to_by" in artwork and len(artwork["referred_to_by"]) > 0 and "content" in artwork["referred_to_by"][0]:  
            text_text = artwork["referred_to_by"][0]["content"]
        
        try:
            if artwork["subject_of"][0]["classified_as"][1]["id"] == "http://vocab.getty.edu/aat/300266277":
                homepage = artwork["subject_of"][0]["id"]
                text_text = text_tex + "<br/><br/><a target='_new' href='" + homepage+ "'>Artwork homepage</a>"
        except:
            a=1

#dates 
        if "begin_of_the_begin" in artwork["produced_by"]["timespan"]:
            start_date_year = artwork["produced_by"]["timespan"]["begin_of_the_begin"]
        if "end_of_the_end" in artwork["produced_by"]["timespan"]:
            end_date_year = artwork["produced_by"]["timespan"]["end_of_the_end"]
    
        display_date = start_date_year

        event = {
            "media": {  "url":media_url,      "caption": media_caption,      "credit": media_credit,      "thumbnail": media_thumbnail },
            "display_date": display_date,
            "start_date": {          "year": start_date_year },        
            "end_date": {         "year":  end_date_year     },
            "text": {       "headline": text_headline,      "text":  text_text     } ,
            "type": "title",
            "background":{"color":background_colour}
        }
  
        json_vis["events"].append(event)


## Define Timeline File Location

In [10]:
timeline_file = "./data/working/timeline.json"

## Write to File

In [11]:
text_file = open(timeline_file, 'wt')  
n = text_file.write(json.dumps(json_vis,indent=2))
text_file.close()

## Visualisation

In [12]:
%%HTML
        <link title="timeline-styles" rel="stylesheet"   href="https://cdn.knightlab.com/libs/timeline3/latest/css/timeline.css">
        <script src="https://cdn.knightlab.com/libs/timeline3/latest/js/timeline.js"></script>
        <div id='timeline-embed2' style="width: 100%; height: 800px"></div>
        <script type="text/javascript">   window.timeline = new TL.Timeline('timeline-embed2', "./data/working/timeline.json");  </script>